In [1]:
from uuid import uuid4
import numpy as np
import scipy.stats as st

In [2]:
np.random.seed(21)

In [3]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, Flatten, Dense, Lambda, Activation
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.losses import mean_squared_error, mean_absolute_error, categorical_crossentropy, binary_crossentropy
from keras.preprocessing.image import Iterator
import matplotlib.pyplot as plt
import cv2
import os
import json
import pickle
%matplotlib inline

from keras.utils.np_utils import to_categorical

from keras.layers import Reshape, BatchNormalization

import tensorflow as tf
sess = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


In [7]:
def get_unet(n_channels=3):
    # Fully convolutional, we don't specify the image size
    inputs = Input((None, None, n_channels))
    
    conv1 = Conv2D(32, (3, 3), activation='elu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='elu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='elu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='elu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='elu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='elu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='elu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='elu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='elu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, (3, 3), activation='elu', padding='same')(conv5)
    
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=-1)
    conv6 = Conv2D(256, (3, 3), activation='elu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='elu', padding='same')(conv6)
    
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = Conv2D(128, (3, 3), activation='elu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='elu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Conv2D(64, (3, 3), activation='elu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='elu', padding='same')(conv8)

    #up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    #conv9 = Conv2D(32, (3, 3), activation='elu', padding='same')(up9)
    #conv9 = Conv2D(32, (3, 3), activation='elu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv8)
    model = Model(inputs=[inputs], outputs=[conv10])

    return model

In [8]:
unet = get_unet()

In [9]:
t = np.ones((1, 512, 512, 3))

In [10]:
unet.predict(t).shape

(1, 256, 256, 1)

In [15]:
t = K.constant(np.random.randint(0, 3, size=(128, 128)))

In [30]:
m = K.cast(K.not_equal(t, 2), K.floatx())

In [31]:
m

<tf.Tensor 'Cast_1:0' shape=(128, 128) dtype=float32>

In [32]:
with sess.as_default():
    print(t.eval())

[[ 1.  0.  0. ...,  2.  2.  1.]
 [ 2.  2.  0. ...,  0.  2.  1.]
 [ 1.  1.  2. ...,  0.  0.  1.]
 ..., 
 [ 0.  2.  1. ...,  1.  1.  1.]
 [ 2.  1.  1. ...,  2.  1.  1.]
 [ 1.  0.  1. ...,  2.  1.  2.]]


In [33]:
with sess.as_default():
    print(m.eval())

[[ 1.  1.  1. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  1.  0.  1.]
 [ 1.  1.  0. ...,  1.  1.  1.]
 ..., 
 [ 1.  0.  1. ...,  1.  1.  1.]
 [ 0.  1.  1. ...,  0.  1.  1.]
 [ 1.  1.  1. ...,  0.  1.  0.]]


In [34]:
with sess.as_default():
    print((t * m).eval())

[[ 1.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 1.  1.  0. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  1. ...,  1.  1.  1.]
 [ 0.  1.  1. ...,  0.  1.  1.]
 [ 1.  0.  1. ...,  0.  1.  0.]]


In [84]:
from keras.losses import hinge

In [86]:
K.binary_crossentropy(y_true, y_pred)

<tf.Tensor 'logistic_loss_12:0' shape=(12, 128, 128) dtype=float32>

In [95]:
def binary_crossentropy_ignore(y_true, y_pred):
    ign_label = 2
    mask = K.cast(K.not_equal(y_true, ign_label), K.floatx())
    loss_per_pixel = K.binary_crossentropy(y_true, y_pred)
    loss_not_ign = loss_per_pixel * mask
    used_pixels = K.sum(mask)
    loss_sum = K.sum(loss_not_ign)
    return loss_sum/used_pixels

In [96]:
y_true = K.constant(np.random.randint(0, 3, size=(12, 128, 128)))
y_pred = K.constant(np.random.randint(0, 3, size=(12, 128, 128)))

In [92]:
loss = binary_crossentropy_ignore(y_true, y_pred)

In [93]:
loss

<tf.Tensor 'truediv_14:0' shape=() dtype=float32>

In [94]:
with sess.as_default():
    print(loss.eval())

8.07217


In [ ]:
class NonValidPatch(Exception):
    pass

In [5]:
def get_block_loc(shape, x, y, target_size=(224, 224), n_blocks=(4,4), overlap=(448,448)):
    h, w = shape
    w_block = (w + (n_blocks[1] - 1) * overlap[1]) // n_blocks[1]
    h_block = (h + (n_blocks[0] - 1) * overlap[0]) // n_blocks[0]
    for by in range(n_blocks[0]):
        y_start = by * (h_block - overlap[0])
        y_end = y_start + h_block + 1
        for bx in range(n_blocks[1]):
            x_start = bx * (w_block - overlap[1])
            x_end = x_start + w_block + 1
            
            if x_start <= x < x_end and y_start <= y < y_end and\
            x_start <= x + target_size[1] - 1 < x_end and y_start <= y + target_size[0] - 1 < y_end:
                return bx + by * n_blocks[0], x - x_start, y - y_start
    raise NonValidPatch("Can't find block...??")

In [7]:
class StreamStats(object):
    """ See https://www.johndcook.com/blog/standard_deviation/
    """
    def __init__(self):
        self.M = None
        self.S = None
        self.k = 0
        self.min = None
        self.max = None
        
    def update(self, x):
        self.k += 1
        if self.k == 1:
            self.M = x
            self.S = 0
            self.min = x
            self.max = x
        else:
            prevM = self.M
            prevS = self.S
            self.M = prevM + (x - prevM)/self.k
            self.S = prevS + (x - prevM) * (x - self.M)
            self.min = np.minimum(x, self.min)
            self.max = np.maximum(x, self.max)
            
    def mean(self):
        return self.M
        
    def variance(self):
        if self.k - 1 > 0:
            return self.S / (self.k - 1)
        else:
            return 0
    
    def std(self):
        return np.sqrt(self.variance())
    
    def minimum(self):
        return self.min
    
    def maximum(self):
        return self.max

In [8]:
from collections import defaultdict

In [37]:
class PatchIterator(Iterator):
    """Iterator yielding training samples
    :param root_dir: Directory containing training images, density map and sampling map.
    :param image_ids: Set of image ids to use to sample patches.
    :param n_samples_per_image: Number of patches to sample on each image.
    :param target_size: Size of the patches sampled.
    :param batch_size: Number of patches sampled per batch
    :param shuffle: Boolean, whether to shuffle the data between epochs.
    :param seed: Random seed for data shuffling.
    :return batch_x, batch_x. 
        batch_x is a (batch_size, target_size[0], target_size[1], 3) array
        batch_x is a (batch_size, target_size[0], target_size[1], 1) array if output_counts is False
        otherwise, it is a (batch_size, 5) array.
    """

    def __init__(self, root_dir, image_ids,
                 class_weights = None,
                 n_samples_per_image=160,
                 target_size=(224, 224),
                 batch_size=8, shuffle=True, seed=42,
                 debug_dir=None):
        
        self.n_sealion_types = 5
        self.image_ids = image_ids
        self.root_dir = root_dir
        self.debug_dir = debug_dir
        
        # Normalize to use class_weights as a probability distribution.
        if class_weights:
            self.class_weights = np.asarray(class_weights)/np.sum(class_weights)
        else:
            self.class_weights = np.ones((self.n_sealion_types+1))/(self.n_sealion_types + 1)
        
        self.normalize = True
        self.n_samples_per_image = n_samples_per_image
        self.target_size = target_size
        self.n_indices = len(self.image_ids) * self.n_samples_per_image
                 
        super(PatchIterator, self).__init__(self.n_indices, batch_size, shuffle, seed)
        
    def load_stats(self, uuid):
        self.x_scale = np.load(os.path.join(self.root_dir, "x_scale_{}.npy".format(uuid)))
        self.x_offset = np.load(os.path.join(self.root_dir, "x_offset_{}.npy".format(uuid)))
        
    def compute_class_distribution(self, n_batches):
        total = 0
        count_per_classes = defaultdict(int)
        for b in range(n_batches):
            _, by = self.next()
            by = np.argmax(by, axis=-1)
            ids, counts = np.unique(by, return_counts=True)
            for i in range(ids.shape[0]):
                count_per_classes[ids[i]] += counts[i]
                total += counts[i]
            
        class_counts = []
        for i in range(self.n_sealion_types + 1):
            class_counts.append(count_per_classes[i])
        return class_counts
            
    def compute_stats(self, n_batches=100):
        self.normalize = False
        uuid = uuid4().hex
        input_stats = StreamStats()
        log_output_stats = StreamStats()
        scaled_output_stats = StreamStats()
        
        for b in range(n_batches):
            bx, _ = self.next()
            for b in range(bx.shape[0]):
                x = bx[b,...]
                x_mean_bgr = np.mean(np.mean(x, axis=0), axis=0)
                input_stats.update(x_mean_bgr)
                    
        self.x_scale = 1/(input_stats.std())
        self.x_offset = -input_stats.mean()/input_stats.std()
        np.save(os.path.join(self.root_dir, "x_scale_{}.npy".format(uuid)), self.x_scale)
        np.save(os.path.join(self.root_dir, "x_offset_{}.npy".format(uuid)), self.x_offset)
        
        self.normalize = True
        return uuid

    def normalize_input(self, x):
        if not self.normalize: return x
        
        return x * self.x_scale + self.x_offset
    
    def denormalize_input(self, x_normed):
        return(x_normed - self.x_offset)/self.x_scale
    
    def random_flip(self, x, y):
        flips = np.random.randint(0, 2, (3,))
        if flips[0]:
            x = np.rot90(x)
            y = np.rot90(y)
        if flips[1]:
            x = np.flipud(x)
            y = np.flipud(y)
        if flips[2]:
            x = np.fliplr(x)
            y = np.fliplr(y)
        return x, y

    def sample(self, shape, dots, image_id):
        # if more than 30% of the patch is masked, reject it
        threshold_masked = 0.3 
        
        # Set probability to 0 if some sealion type is not in the block
        current_weigths = self.class_weights.copy()
        for i in range(self.n_sealion_types):
            if not dots[i]:
                current_weigths[i] = 0
        current_weigths /= np.sum(current_weigths)

        while 1:
            # Choose an output class randomly
            output_class = np.random.choice(self.n_sealion_types + 1, size=(1, ), p=current_weigths)[0]

            try:
                # Sample a location, either for background or for a sealion.
                if output_class == self.n_sealion_types:
                    x, y = self.sample_bg(shape, image_id)
                else:
                    x, y = self.sample_dot(shape, dots[output_class], image_id)
            
                # Get the corresponding image block, and (x, y) in this block
                bid, x, y = get_block_loc(shape, x, y)
            except NonValidPatch:
                continue
                
            uid = "{iid}_{bid}".format(iid=image_id, bid=bid)
            img = cv2.imread(os.path.join(self.root_dir, "TrainBlock", uid + ".jpg"))
            img_patch = img[y:y+self.target_size[0], x:x+self.target_size[1],:]
            masked_pixels = np.count_nonzero(img_patch == 0)
            total_pixels = img_patch.shape[0] * img_patch.shape[1]
            if img_patch.shape[0] != self.target_size[0] or img_patch.shape[1] != self.target_size[1]:
                continue
            if masked_pixels/total_pixels < threshold_masked:
                smap = np.load(os.path.join(self.root_dir, "TrainSegmentation", uid + ".npz"))['smap']
                smap_patch = smap[y:y+self.target_size[0], x:x+self.target_size[1]]
                return self.random_flip(img_patch, smap_patch)
        
    def sample_bg(self, shape, image_id):
        x = np.random.randint(0, shape[1] - self.target_size[1], size=(1,))[0]
        y = np.random.randint(0, shape[0] - self.target_size[0], size=(1,))[0]
        
        return x, y
    
    def sample_dot(self, shape, dots, image_id):
        half_size = 40
        
        randi = np.random.choice(len(dots), size=(1,))[0]
        rand_dot = dots[randi]
        
        min_x = max(0, rand_dot[0] - self.target_size[1] + half_size)
        max_x = min(shape[1] - self.target_size[1], rand_dot[0] + self.target_size[1] - half_size)
        
        min_y = max(0, rand_dot[1] - self.target_size[0] + half_size)
        max_y = min(shape[0] - self.target_size[0], rand_dot[1] + self.target_size[0] - half_size)
        
        if min_x > max_x:
            max_x, min_x = min_x, max_x
        if min_y > max_y:
            max_y, min_y = min_y, max_y 
            
        if min_x == max_x or min_y == max_y:
            raise NonValidPatch()
           
        x = np.random.randint(min_x, max_x, size=(1,))[0]
        y = np.random.randint(min_y, max_y, size=(1,))[0]
        
        return x, y
        
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
                 
        batch_x = np.zeros((current_batch_size, self.target_size[0], self.target_size[1], 3), dtype=K.floatx())
        batch_y = np.zeros((current_batch_size, self.target_size[0], self.target_size[1], self.n_sealion_types + 1), dtype=K.floatx())
        
        # For each index, we load the data and sample randomly n_successive_samples patches
        for i, j in enumerate(index_array):
            index = j // self.n_samples_per_image
            image_id = self.image_ids[index]
            with open(os.path.join(self.root_dir, "TrainDots", str(image_id) + ".pkl"), "rb") as pfile:
                dots = pickle.load(pfile)
            with open(os.path.join(self.root_dir, "TrainShape", str(image_id) + ".pkl"), "rb") as pfile:
                shape = pickle.load(pfile)
                
            x, y = self.sample(shape, dots, image_id)
            batch_x[i,:,:,:] = x
            batch_y[i,:,:,:] = to_categorical(y).reshape(y.shape[:2] + (self.n_sealion_types + 1, ))    

        if self.debug_dir:
            for i in range(current_batch_size):
                cv2.imwrite(os.path.join(self.debug_dir, "patch_{}.jpg".format(i)), batch_x[i])
                
        return self.normalize_input(batch_x), batch_y

In [38]:
mis_matched = set()
with open("../data/sealion/real_missmatched", "r") as f:
    f.readline()
    for line in f:
        mis_matched.add(int(line))

with open("../data/sealion/train.json", "r") as jfile:
    train_ids = json.load(jfile)
train_ids = [int(s[:-4]) for s in train_ids]

train_ids = list(set(train_ids).difference(mis_matched))

with open("../data/sealion/val.json", "r") as jfile:
    val_ids = json.load(jfile)
val_ids = [int(s[:-4]) for s in val_ids]
val_ids = list(set(val_ids).difference(mis_matched))

In [42]:
class_weights = [0.34,  0.42, 0.05, 0.09, 0.11, 1/10]

In [43]:
trainPatchesGenerator = PatchIterator("/home/lowik/sealion/data/sealion/", train_ids, class_weights)

In [ ]:
uuid = trainPatchesGenerator.compute_stats(100)

In [44]:
uuid = 'b21a839a057b447991fc8e9cfb46cf12'

In [45]:
trainPatchesGenerator.load_stats(uuid)

In [46]:
valPatchesGenerator = PatchIterator("/home/lowik/sealion/data/sealion/", val_ids, class_weights)
valPatchesGenerator.load_stats(uuid)

In [ ]:
classes_counts = trainPatchesGenerator.compute_class_distribution(100)

In [ ]:
mcount = np.median(classes_counts)

In [ ]:
weights_per_class = mcount/classes_counts

In [ ]:
weights_per_class

In [ ]:
weights_per_class_clipped = np.clip(weights_per_class, 0.02, 5)

In [ ]:
np.sum(classes_counts * weights_per_class_clipped)

In [ ]:
np.sum(classes_counts)

In [ ]:
40140800/3500186

In [ ]:
weights_per_class_clipped *= 5

In [ ]:
weights_per_class_clipped

In [ ]:
weights_per_class_clipped = np.array([  7.44099153,   9.65796137,   2.47413777,   3.7649283 ,
        15.        ,   0.1       ])

In [ ]:
np.sum(classes_counts * np.ones((6,)))

In [ ]:
np.sum(classes_counts * weights_per_class_clipped)

In [47]:
weights_per_class_clipped = np.array([7, 9, 5, 6, 10, 0.1])

In [48]:
def w_categorical_crossentropy(y_true, y_pred):
    y_true_r = K.reshape(y_true, (8, 224*224, 6))
    y_pred_r = K.reshape(y_pred, (8, 224*224, 6))
    weights = K.max(y_true_r * weights_per_class_clipped, axis=-1)
    loss_per_pixel = K.categorical_crossentropy(y_pred_r, y_true_r)
    return loss_per_pixel * weights

In [49]:
unet = get_unet(224, 224, 3, 6, batch_norm=True)
sgd = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)
unet.compile(optimizer=sgd, loss=w_categorical_crossentropy, metrics=["accuracy"])

In [50]:
h = unet.fit_generator(trainPatchesGenerator, 250, epochs=5, verbose=1, callbacks=None, validation_data=valPatchesGenerator, validation_steps=75, class_weight=None, max_q_size=2, workers=1, pickle_safe=False, initial_epoch=0)

Epoch 1/5
215/250 [========================>.....] - ETA: 573s - loss: 0.7751 - acc: 0.5752

KeyboardInterrupt: 

In [ ]:
bpred = unet.predict(batch_x)

In [ ]:
i = 0
plt.figure(figsize=(15, 5))
plt.subplot(121)
plt.imshow(valPatchesGenerator.denormalize_input(batch_x[i,:,:, :]))
plt.subplot(122)
plt.imshow(np.argmax(bpred[i,...], axis=-1))

In [ ]:
unet.save("../data/unet_segmentation_ellipse_dmap_sgd_10epochs_200steps.h5")

In [ ]:
for batch_x, batch_y in valPatchesGenerator:
    break

In [ ]:
batch_ypred = unet.predict(batch_x)

In [ ]:
batch_ypred.shape

In [ ]:
np.min(batch_ypred[:,:,2])

In [ ]:
gg = np.argmax(batch_ypred, axis=-1)

In [ ]:
np.unique(gg, return_counts=True)

In [ ]:
i = 0
plt.figure(figsize=(15, 5))
plt.subplot(131)
plt.imshow(valPatchesGenerator.denormalize_input(batch_x[i,:,:, :]))
plt.subplot(132)
plt.imshow(np.argmax(batch_y[i,...], axis=-1))
plt.subplot(133)
plt.imshow(np.argmax(batch_ypred[i,...], axis=-1))

In [ ]:
plt.imshow(valPatchesGenerator.denormalize_output(batch_ypred[i,:,:, 0]) > 0.0007)

In [ ]:
i = 7
print("GT: ", np.sum(valPatchesGenerator.denormalize_output(batch_y[i,:,:, 0])))
print("Pred: ", np.sum(valPatchesGenerator.denormalize_output(batch_ypred[i,:,:, 0])))

In [ ]:
def full_image_process(im, net, patchGenerator, patch_size=(224, 224), batch_size=8, overlap=(64, 64)):
    h, w, c = im.shape
    n_patches_x = int(np.ceil((w - patch_size[1])/(patch_size[1] - overlap[1]) + 1))
    n_patches_y = int(np.ceil((h - patch_size[0])/(patch_size[0] - overlap[0]) + 1))
    print(n_patches_x, n_patches_x)
    
    dmap = np.zeros((h, w, 1), dtype=np.float32)
    dmap_count = np.zeros((h, w, 1), dtype=np.int8)
    batch_x = np.zeros((batch_size, ) + patch_size + (c, ), dtype=np.float32)
    batch_pos = np.zeros((batch_size, 4), dtype=np.int32)
    
    current_batch_size = 0
    for py in range(n_patches_y):
        y_start = py * (patch_size[0] - overlap[0])
        y_start = min(h - patch_size[0], y_start)
        y_end = y_start + patch_size[0]
        for px in range(n_patches_x):
            x_start = px * (patch_size[1] - overlap[1])
            x_start = min(w - patch_size[1], x_start)
            x_end = x_start + patch_size[1]
            
            # Keep filling the batch
            batch_x[current_batch_size, :, :, :] = im[y_start:y_end, x_start:x_end, :]
            batch_pos[current_batch_size, :] = np.array([y_start, y_end, x_start, x_end])
            current_batch_size += 1
            
            if current_batch_size == batch_size or (py == n_patches_y - 1 and px == n_patches_x - 1) :
                # time to predict
                batch_x_normed = patchGenerator.normalize_input(batch_x)
                batch_ylog = net.predict(batch_x_normed)
                batch_y = patchGenerator.denormalize_output(batch_ylog)
                # Fill the full dmap
                for i in range(current_batch_size):
                    y_start, y_end, x_start, x_end = tuple(batch_pos[i,:])
                    dmap[y_start:y_end, x_start:x_end, :] += batch_y[i,:,:,:]
                    dmap_count[y_start:y_end, x_start:x_end] += 1
                current_batch_size = 0
                
    return dmap, dmap_count

In [ ]:
im = cv2.imread("../data/sealion/Train/872.jpg")

In [ ]:
dmap, dmap_count = full_image_process(im, unet, valPatchesGenerator)

In [ ]:
plt.imshow(dmap[:,:,0])

In [ ]:
dmap_avg = dmap/dmap_count

In [ ]:
dmap_count.shape

In [ ]:
np.sum(dmap)

In [ ]:
np.sum(dmap_avg)

In [ ]:
dmap_gt = np.load("../data/sealion/TrainDensity/872_0.npz")